# OpenVINO benchmarking with InceptionV3
In this tutorial, we will use the Intel® Distribution of OpenVINO™ Toolkit to perform benchmarking on InceptionV3 model.

This tutorial assumes that you have already downloaded and installed [Intel&reg; OpenVINO&trade;](https://software.intel.com/en-us/openvino-toolkit/choose-download) on your computer. 

In order to use Intel® OpenVINO™, we need to do a few steps:

1. Convert our Keras model to a Tensorflow model. 
1. Freeze the Tensorflow saved format model
1. Use the OpenVINO Model Optimizer to convert the above freezed-model to the OpenVINO Intermediate Representation (IR) format
1. Benchmark using the OpenVINO benchmark tool: `/opt/intel/openvino/deployment_tools/tools/benchmark_tool/benchmark_app.py`


In [1]:
import keras
import os
import tensorflow as tf

Using TensorFlow backend.


In [2]:
model = keras.applications.inception_v3.InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)














96116736/96112376 [==============================] - 1s 0us/step


In [3]:
sess = keras.backend.get_session()
sess.run(tf.global_variables_initializer())

In [4]:
output_directory = "/home/ubuntu/models/inceptionv3/output/"
print("Freezing the graph.")
keras.backend.set_learning_phase(0)

signature = tf.saved_model.signature_def_utils.predict_signature_def(
    inputs={'input': model.input}, outputs={'output': model.output})

#If directory exists, delete it and let builder rebuild the TF model.
if os.path.isdir(output_directory):
    print (output_directory, "exists already. Deleting the folder")
    shutil.rmtree(output_directory)

builder = tf.saved_model.builder.SavedModelBuilder(output_directory)
builder.add_meta_graph_and_variables(sess=sess,    
                                     tags=[tf.saved_model.tag_constants.SERVING],    
                                     signature_def_map={
                                         tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:signature
                                     })
builder.save() 
print("TensorFlow protobuf version of model is saved in:", output_directory)

print("Model input name = ", model.input.op.name)
print("Model input shape = ", model.input.shape)
print("Model output name = ", model.output.op.name)
print("Model output shape = ", model.output.shape)

Freezing the graph.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /home/ubuntu/models/inceptionv3/output/saved_model.pb
TensorFlow protobuf version of model is saved in: /home/ubuntu/models/inceptionv3/output/
Model input name =  input_1
Model input shape =  (?, 299, 299, 3)
Model output name =  predictions/Softmax
Model output shape =  (?, 1000)


In [5]:
output_frozen_model_dir = "/home/ubuntu/models/inceptionv3/frozen_model"
output_frozen_graph = output_frozen_model_dir+'/saved_model_frozen.pb'

if not os.path.isdir(output_frozen_model_dir):
    os.mkdir(output_frozen_model_dir)
else:
    print('Directory', output_frozen_model_dir, 'already exists. Deleting it and re-creating it')
    shutil.rmtree(output_frozen_model_dir)
    os.mkdir(output_frozen_model_dir)

from tensorflow.python.tools.freeze_graph import freeze_graph

_ = freeze_graph(input_graph="",
             input_saver="",
             input_binary=False,
             input_checkpoint="",
             restore_op_name="save/restore_all",
             filename_tensor_name="save/Const:0",
             clear_devices=True,
             initializer_nodes="",
             input_saved_model_dir=output_directory,
             output_node_names=model.output.op.name,
             output_graph=output_frozen_graph)

print("TensorFlow Frozen model model is saved in:", output_frozen_graph)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /home/ubuntu/models/inceptionv3/output/variables/variables
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 378 variables.
INFO:tensorflow:Converted 378 variables to const ops.
TensorFlow Frozen model model is saved in: /home/ubuntu/models/inceptionv3/frozen_model/saved_model_frozen.pb


In [6]:
output_frozen_model_dir = "/home/ubuntu/models/inceptionv3/frozen_model"
output_frozen_graph = output_frozen_model_dir+'/saved_model_frozen.pb'

if not os.path.exists(output_frozen_graph):
    print(output_frozen_graph + ' doesn\'t exist. Please make sure you have a trained keras to TF frozen model')

!mo_tf.py \
      --input_model '/home/ubuntu/models/inceptionv3/frozen_model/saved_model_frozen.pb' \
      --input_shape=[1,299,299,3] \
      --data_type FP32  \
      --output_dir /home/ubuntu/models/inceptionv3/IR_models/FP32  \
      --model_name saved_model


Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/ubuntu/models/inceptionv3/frozen_model/saved_model_frozen.pb
	- Path for generated IR: 	/home/ubuntu/models/inceptionv3/IR_models/FP32
	- IR output name: 	saved_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,299,299,3]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration file with input/output no

## Benchmark

Benchmark using the following command:
```
python3 /opt/intel/openvino/deployment_tools/tools/benchmark_tool/benchmark_app.py \
-m /home/ubuntu/models/inceptionv3/IR_models/FP32/saved_model.xml \
-nireq 1 -nstreams 1
```

In [ ]:
!python3 /opt/intel/openvino/deployment_tools/tools/benchmark_tool/benchmark_app.py \
-m /home/ubuntu/models/inceptionv3/IR_models/FP32/saved_model.xml \
-nireq 1 -nstreams 1

#### Sample Output
```
[Step 1/11] Parsing and validating input arguments
[Step 2/11] Loading Inference Engine
[ INFO ] InferenceEngine:
         API version............. 2.1.37988
[ INFO ] Device info
         CPU
         MKLDNNPlugin............ version 2.1
         Build................... 37988

[Step 3/11] Reading the Intermediate Representation network
[Step 4/11] Resizing network to match image sizes and given batch
[ INFO ] Network batch size: 1, precision: MIXED
[Step 5/11] Configuring input of the model
[Step 6/11] Setting device configuration
[Step 7/11] Loading the model to the device
[Step 8/11] Setting optimal runtime parameters
[Step 9/11] Creating infer requests and filling input blobs with images
[ INFO ] Network input 'input_1' precision U8, dimensions (NCHW): 1 3 299 299
[ WARNING ] No input files were given: all inputs will be filled with random values!
[ INFO ] Infer Request 0 filling
[ INFO ] Fill input 'input_1' with random values (image is expected)
[Step 10/11] Measuring performance (Start inference asyncronously, 1 inference requests using 1 streams for CPU, limits: 60000 ms duration)
[Step 11/11] Dumping statistics report
Count:      8279 iterations
Duration:   60030.59 ms
Latency:    6.93 ms
Throughput: 137.91 FPS
```